In [6]:
import pandas as pd
import os
import ast

In [10]:
df_movies = pd.read_csv(r'../movies_dataset.csv') # Lectura de primer dataset
df_new = df_movies.copy()

C:\Users\Jesus Manuel Martine\AppData\Local\Temp\ipykernel_10604\2163314.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv(r'../movies_dataset.csv') # Lectura de primer dataset


# Desanidando DataFrame

In [20]:
# Desanidar y dividir en belongs_to_collection(id,name)
def desanidar_belongs_to_collection(df, column_name='belongs_to_collection'):
    def desanidar_collection(collection):
        if pd.isna(collection):
            return {"id": None, "name": None}
        try:
            collection_dict = ast.literal_eval(collection)
            if isinstance(collection_dict, dict):
                return {"id": collection_dict.get("id"), "name": collection_dict.get("name")}
            else:
                return {"id": None, "name": None}
        except (ValueError, SyntaxError):
            return {"id": None, "name": None}
    
    # Aplicar la función a la columna y crear nuevas columnas para 'id' y 'name'
    collection_data = df[column_name].apply(desanidar_collection)
    collection_df = pd.json_normalize(collection_data)
    
    # Renombrar columnas para mayor claridad
    collection_df.columns = ['belongs_to_collection_id', 'belongs_to_collection_name']
    
    # Unir las nuevas columnas al dataframe original y eliminar la columna original
    df = pd.concat([df, collection_df], axis=1)
    df = df.drop(columns=[column_name])
    
    return df
df_new = desanidar_belongs_to_collection(df_new,column_name='belongs_to_collection')

In [21]:
df_new['release_date'] = pd.to_datetime(df_new['release_date'], format = "%Y-%m-%d" , errors='coerce')
df_new["release_date"].dt.year
df_new = df_new.dropna(subset="release_date")

df_new["release_year"] = df_new["release_date"].dt.year
df_new["release_year"] = df_new["release_year"].astype(int)

In [22]:
def buscar_nombre(lista):
    #en este caso esto convierte los objetos a listas
    lista = ast.literal_eval(lista)
    # en caso de haber mas de un diccionario con la clave name devuelve todos los valores concatenados
    return ', '.join([i['name'] for i in lista])

df_new['genres'] = df_new['genres'].fillna('[]').apply(buscar_nombre)
df_new['spoken_languages'] = df_new['spoken_languages'].fillna('[]').apply(buscar_nombre)
df_new['production_countries'] = df_new['production_countries'].fillna('[]').apply(buscar_nombre)
df_new['production_companies'] = df_new['production_companies'].fillna('[]').apply(buscar_nombre)


In [23]:
df_new[["budget", "revenue"]].fillna(0, inplace = True)
df_new= df_new.dropna(subset=["release_date"])

C:\Users\Jesus Manuel Martine\AppData\Local\Temp\ipykernel_240\170581034.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[["budget", "revenue"]].fillna(0, inplace = True)


In [24]:
df_new["budget"] = df_new["budget"].astype(float)
df_new['return'] = df_new['revenue'] / df_new['budget'].replace(0, 1)
df_new.drop(columns = ["video","imdb_id", "original_title","poster_path" ,"homepage"],inplace = True)

,adult,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,belongs_to_collection_id,belongs_to_collection_name,release_year,return
0,False,30000000.0,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,...,English,Released,NaN,Toy Story,7.7,5415.0,10194.0,Toy Story Collection,1995,1.245180e+01
1,False,65000000.0,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,1995-12-15,...,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,None,1995,4.043035e+00
2,False,0.0,"Romance, Comedy",15602,en,A family wedding reignites the ancient feud be...,11.7129,"Warner Bros., Lancaster Gate",United States of America,1995-12-22,...,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,119050.0,Grumpy Old Men Collection,1995,0.000000e+00
3,False,16000000.0,"Comedy, Drama, Romance",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,...,English,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,NaN,None,1995,5.090760e+00
4,False,0.0,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,"Sandollar Productions, Touchstone Pictures",United States of America,1995-02-10,...,English,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,96871.0,Father of the Bride Collection,1995,7.657891e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,False,0.0,"Drama, Action, Romance",30840,en,"Yet another version of the classic epic, with ...",5.683753,"Westdeutscher Rundfunk (WDR), Working Title Fi...","Canada, Germany, United Kingdom, United States...",1991-05-13,...,English,Released,NaN,Robin Hood,5.7,26.0,NaN,None,1991,0.000000e+00
45462,False,0.0,Drama,111109,tl,An artist struggles to finish his work while a...,0.178241,Sine Olivia,Philippines,2011-11-17,...,,Released,NaN,Century of Birthing,9.0,3.0,NaN,None,2011,0.000000e+00
45463,False,0.0,"Action, Drama, Thriller",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,American World Pictures,United States of America,2003-08-01,...,English,Released,A deadly game of wits.,Betrayal,3.8,6.0,NaN,None,2003,0.000000e+00
45464,False,0.0,,227506,en,"In a small town live two brothers, one a minis...",0.003503,Yermoliev,Russia,1917-10-21,...,,Released,NaN,Satan Triumphant,0.0,0.0,NaN,None,1917,0.000000e+00


# Comienzo de Credits

In [12]:
df_credits = pd.read_csv('../credits.csv')
df_credits_cast = df_credits[['cast']].copy()
df_credits_crew = df_credits[['crew']].copy()


In [13]:
# Desanidado de Cast,Crew y limpieza
def desanidar(df, column_name='cast'):
    def desanidar_list(list):
        if pd.isna(list):
            return []
        try:
            return ast.literal_eval(list)
        except (ValueError, SyntaxError):
            return []
    
   
    desanidado = df[column_name].apply(desanidar_list)
    desanidado_df = desanidado.explode().reset_index(drop=True)

    cast_df = pd.json_normalize(desanidado_df)
    
    return cast_df
df_cast_limpio = desanidar(df_credits_cast,column_name='cast')
df_crew_limpio = desanidar(df_credits_crew,column_name='crew')


In [14]:
df_cast_limpio = df_cast_limpio.rename(columns={
    'character': 'Voces',
    'id': 'actor_id',
    'name': 'name_actor',
})

In [15]:
def guardar_csv_en_carpeta(df, nombre_archivo, carpeta='Data'):

    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    ruta_archivo = os.path.join(carpeta, nombre_archivo)

    df.to_csv(ruta_archivo, index=False)

In [16]:
# Cargando Dataframes Limpios
guardar_csv_en_carpeta(df_cast_limpio[['cast_id','actor_id','name_actor','credit_id']],'cast_limpio.csv',carpeta='Data')
guardar_csv_en_carpeta(df_crew_limpio[['credit_id','id','job','name']],'crew_limpio.csv',carpeta='Data')
guardar_csv_en_carpeta(df_new[['title','release_date','vote_count','vote_average']],'Datos_de_Pelicula.csv',carpeta='Data')